In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tkinter as tk
from tkinter import Label, messagebox
from PIL import Image, ImageTk
import joblib



In [17]:
def center_window(win):
    win.update_idletasks()
    width = win.winfo_width()
    frm_width = win.winfo_rootx() - win.winfo_x()
    win_width = width + 2 * frm_width
    height = win.winfo_height()
    titlebar_height = win.winfo_rooty() - win.winfo_y()
    win_height = height + titlebar_height + frm_width
    x = win.winfo_screenwidth() // 2 - win_width // 2
    y = win.winfo_screenheight() // 2 - win_height // 2
    win.geometry('{}x{}+{}+{}'.format(width, height, x, y))
    win.deiconify()

def display_logo():
    logo_path = r'C:\Users\data science\Downloads\GitHub_Repositories\OTT_SUGGESTER\images\cinemalytics_nobackground.png'
    img = Image.open(logo_path)
    img = img.resize((100, 100), Image.Resampling.LANCZOS)
    logo_image = ImageTk.PhotoImage(img)
    logo_label = Label(app, image=logo_image)
    logo_label.image = logo_image
    logo_label.pack(pady=20)

model = None

def load_model():
    global clf, label_encoder_genre, label_encoder_age_rating, label_encoder_platform
    clf = load(r'C:\Users\data science\Desktop\OTT_project\trained_models\random_forest_model.joblib')
    label_encoder_genre = load(r'C:\Users\data science\Desktop\OTT_project\trained_models\genre_encoder.joblib')
    label_encoder_age_rating = load(r'C:\Users\data science\Desktop\OTT_project\trained_models\age_rating_encoder.joblib')
    label_encoder_platform = load(r'C:\Users\data science\Desktop\OTT_project\trained_models\platform_encoder.joblib')


def show_recommendation():
    clear_window()
    display_logo()

    # Correctly gather inputs from the GUI
    genre_input = genre_entry.get()
    duration_input = length_entry.get()
    age_rating_input = rating_entry.get()

    # Input validation and preprocessing
    try:
        # Ensure duration is an integer
        duration_input = int(duration_input)
        # Transform inputs using the label encoders
        genre_encoded = label_encoder_genre.transform([genre_input])[0]
        age_rating_encoded = label_encoder_age_rating.transform([age_rating_input])[0]
    except ValueError as e:
        messagebox.showerror("Input Error", f"An error occurred: {e}\nPlease ensure your inputs are valid and try again.")
        return
    
    # Prepare the inputs in the format expected by the model
    new_movie = pd.DataFrame({
        'genre_encoded': [genre_encoded],
        'duration_min': [duration_input],
        'age_rating_encoded': [age_rating_encoded]
    })

    # Make prediction using the model
    try:
        predicted_platform_encoded = clf.predict(new_movie)
        predicted_platform = label_encoder_platform.inverse_transform(predicted_platform_encoded)
    except Exception as e:
        messagebox.showerror("Prediction Error", f"An error occurred during prediction: {e}")
        return

    # Display the prediction result
    tk.Label(app, text="Based on your criteria, we recommend:", pady=10).pack()
    #tk.Label(app, text=f"1. {predicted_platform[0]}", pady=10, font=('Helvetica', 16, 'bold')).pack()
    tk.Label(app, text=predicted_platform[0], pady=10, font=('Helvetica', 16, 'bold')).pack()

    # Buttons for navigation
    tk.Button(app, text="Back to Start page", command=welcome_window, padx=20, pady=10).pack(side=tk.LEFT, padx=(100, 10), pady=(10, 20))
    tk.Button(app, text="Close", command=app.destroy, padx=20, pady=10).pack(side=tk.RIGHT, padx=(10, 100), pady=(10, 20))



# Global variables for entry widgets
# genre_entry = None
# length_entry = None
# rating_entry = None




def input_window(content_type):
    global genre_entry, length_entry, rating_entry  # Use global declaration to modify global variables

    clear_window()
    display_logo()

    tk.Label(app, text=content_type, pady=10).pack()
    
    tk.Label(app, text="Genre").pack()
    genre_entry = tk.Entry(app, textvariable=genre_var)  # Initialize the Entry widget
    genre_entry.pack()

    length_label_text = "Movie length (minutes)" if content_type == "Movies" else "Seasons"
    tk.Label(app, text=length_label_text).pack()
    length_entry = tk.Entry(app, textvariable=duration_var)  # Initialize the Entry widget
    length_entry.pack()

    tk.Label(app, text="Rating (e.g., PG, PG-13)").pack()
    rating_entry = tk.Entry(app, textvariable=age_rating_var)  # Initialize the Entry widget
    rating_entry.pack()

    tk.Button(app, text="Enter", command=show_recommendation).pack(pady=20)


def content_selection_window():
    clear_window()
    display_logo()
    tk.Button(app, text="Movie?", command=lambda: input_window("Movies"), padx=20, pady=10).pack(pady=10)
    tk.Button(app, text="TV Show?", command=lambda: input_window("TV Shows"), padx=20, pady=10).pack(pady=10)

def starting_app_window():
    clear_window()
    display_logo()
    tk.Label(app, text="Loading Cinemalytics!", pady=10).pack()
    load_model()
    clear_window()
    display_logo()
    tk.Label(app, text="Loading Complete", pady=10).pack()
    tk.Label(app, text="Press Continue to Start Cinemalytics", pady=10).pack()
    tk.Button(app, text="Continue", command=welcome_window, padx=20, pady=10).pack()
    tk.Label(app, text="© 2024 Cinemalytics, Inc.", pady=10).pack(side=tk.BOTTOM)

def welcome_window():
    clear_window()
    display_logo()
    tk.Label(app, text="Welcome to Cinemalytics!", pady=10).pack()
    tk.Button(app, text="Start", command=content_selection_window, padx=20, pady=10).pack()
    tk.Label(app, text="© 2024 Cinemalytics, Inc.", pady=10).pack(side=tk.BOTTOM)
    #load_model()

def clear_window():
    for widget in app.winfo_children():
        widget.destroy()

def on_close():
    if messagebox.askokcancel("Quit", "Do you want to quit?"):
        app.destroy()



app = tk.Tk()
app.title("Cinemalytics - Platform Recommender")
app.geometry('400x400')
center_window(app)

app.protocol("WM_DELETE_WINDOW", on_close)  # Handle the window close event


genre_var = tk.StringVar()
duration_var = tk.IntVar()
age_rating_var = tk.StringVar()
# result_var = tk.StringVar()

#starting_app_window()
welcome_window()
app.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Program Files\Python312\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\data science\AppData\Local\Temp\ipykernel_2548\104013061.py", line 38, in show_recommendation
    genre_input = genre_entry.get()
                  ^^^^^^^^^^^^^^^^^
  File "c:\Program Files\Python312\Lib\tkinter\__init__.py", line 3127, in get
    return self.tk.call(self._w, 'get')
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
_tkinter.TclError: invalid command name ".!entry"


In [9]:
import tkinter as tk
from tkinter import messagebox, Label
from PIL import Image, ImageTk
#from sklearn.externals import joblib  # Use this or the next line depending on your sklearn version
import joblib
import pandas as pd

# Function to center the application window on the screen
def center_window(win):
    win.update_idletasks()
    width = win.winfo_width()
    frm_width = win.winfo_rootx() - win.winfo_x()
    win_width = width + 2 * frm_width
    height = win.winfo_height()
    titlebar_height = win.winfo_rooty() - win.winfo_y()
    win_height = height + titlebar_height + frm_width
    x = win.winfo_screenwidth() // 2 - win_width // 2
    y = win.winfo_screenheight() // 2 - win_height // 2
    win.geometry('{}x{}+{}+{}'.format(width, height, x, y))
    win.deiconify()

# Function to display the logo in the application window
def display_logo():
    logo_path = r'C:\Users\data science\Downloads\GitHub_Repositories\OTT_SUGGESTER\images\cinemalytics_nobackground.png'
    img = Image.open(logo_path)
    img = img.resize((100, 100), Image.Resampling.LANCZOS)
    global logo_image  # Make logo_image accessible globally or at least outside of this function
    logo_image = ImageTk.PhotoImage(img)
    logo_label = Label(app, image=logo_image)
    logo_label.image = logo_image  
    logo_label.pack(pady=20)

# Function to load the machine learning model and label encoders
def load_model():
    global clf, label_encoder_genre, label_encoder_age_rating, label_encoder_platform
    # Correctly use joblib.load to load your model and encoders
    clf = joblib.load(r'C:\Users\data science\Desktop\OTT_project\trained_models\random_forest_model.joblib')
    label_encoder_genre = joblib.load(r'C:\Users\data science\Desktop\OTT_project\trained_models\genre_encoder.joblib')
    label_encoder_age_rating = joblib.load(r'C:\Users\data science\Desktop\OTT_project\trained_models\age_rating_encoder.joblib')
    label_encoder_platform = joblib.load(r'C:\Users\data science\Desktop\OTT_project\trained_models\platform_encoder.joblib')
    

def input_window(content_type):
    global genre_entry, length_entry, rating_entry  # Use global declaration to modify global variables

    clear_window()
    display_logo()

    tk.Label(app, text=content_type, pady=10).pack()
    
    tk.Label(app, text="Genre").pack()
    genre_entry = tk.Entry(app, textvariable=genre_var)  # Initialize the Entry widget
    genre_entry.pack()

    length_label_text = "Movie length (minutes)" if content_type == "Movies" else "Seasons"
    tk.Label(app, text=length_label_text).pack()
    length_entry = tk.Entry(app, textvariable=duration_var)  # Initialize the Entry widget
    length_entry.pack()

    tk.Label(app, text="Rating (e.g., PG, PG-13)").pack()
    rating_entry = tk.Entry(app, textvariable=age_rating_var)  # Initialize the Entry widget
    rating_entry.pack()

    tk.Button(app, text="Enter", command=show_recommendation).pack(pady=20)


def welcome_window():
    clear_window()
    display_logo()
    tk.Label(app, text="Welcome to Cinemalytics!", pady=10).pack()
    tk.Button(app, text="Start", command=content_selection_window, padx=20, pady=10).pack()
    tk.Label(app, text="© 2024 Cinemalytics, Inc.", pady=10).pack(side=tk.BOTTOM)
    #load_model()

def content_selection_window():
    clear_window()
    display_logo()
    tk.Button(app, text="Movie?", command=lambda: input_window("Movies"), padx=20, pady=10).pack(pady=10)
    tk.Button(app, text="TV Show?", command=lambda: input_window("TV Shows"), padx=20, pady=10).pack(pady=10)

def starting_app_window():
    clear_window()
    display_logo()
    tk.Label(app, text="Loading Cinemalytics!", pady=10).pack()
    load_model()
    clear_window()
    display_logo()
    tk.Label(app, text="Loading Complete", pady=10).pack()
    tk.Label(app, text="Press Continue to Start Cinemalytics", pady=10).pack()
    tk.Button(app, text="Continue", command=welcome_window, padx=20, pady=10).pack()
    tk.Label(app, text="© 2024 Cinemalytics, Inc.", pady=10).pack(side=tk.BOTTOM)


# Function to clear the application window
def clear_window():
    for widget in app.winfo_children():
        widget.destroy()

# Function to handle the window close event
def on_close():
    if messagebox.askokcancel("Quit", "Do you want to quit?"):
        app.destroy()

# Initialize the main application window
app = tk.Tk()
app.title("Cinemalytics - Platform Recommender")
app.geometry('400x400')
center_window(app)
app.protocol("WM_DELETE_WINDOW", on_close)

# Function to display recommendations based on user inputs
def show_recommendation():
    clear_window()
    display_logo()

    try:
        # If you're using textvariable for entries, you could alternatively do: genre_input = genre_var.get()
        genre_input = genre_entry.get()
        duration_input = length_entry.get()
        age_rating_input = rating_entry.get()
    except tk.TclError as e:
        print("One or more widgets have been destroyed or do not exist:", e)
        return  # Exit the function or handle the error appropriately

    # Preprocess and predict
    try:
        # Preprocessing steps (encoding) should go here
        # For demonstration purposes, let's assume it's directly usable
        prediction_input = [[genre_input, duration_input, age_rating_input]]  # Simplified
        predicted_platform = clf.predict(prediction_input)[0]  # Simplified prediction
        
        tk.Label(app, text="Based on your criteria, we recommend:", pady=10).pack()
        tk.Label(app, text=predicted_platform, pady=10, font=('Helvetica', 16, 'bold')).pack()
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")
    
    # Buttons for navigation
    tk.Button(app, text="Back to Start page", command=welcome_window, padx=20, pady=10).pack(side=tk.LEFT, padx=(100, 10), pady=(10, 20))
    tk.Button(app, text="Close", command=app.destroy, padx=20, pady=10).pack(side=tk.RIGHT, padx=(10, 100), pady=(10, 20))

# Various window setup functions (simplified for brevity)

genre_var = tk.StringVar()
duration_var = tk.IntVar()
age_rating_var = tk.StringVar()

# Start the application
#load_model()  # Load your model and label encoders
#display_logo()  # Initially display logo and wait for user interaction
starting_app_window()  # Display the welcome window

app.mainloop()


One or more widgets have been destroyed or do not exist: invalid command name ".!entry"


In [21]:
import tkinter as tk
from joblib import load
import pandas as pd

# Load the trained model and label encoders from their saved locations
clf = load(r'C:\Users\data science\Desktop\OTT_project\trained_models\random_forest_model.joblib')
label_encoder_genre = load(r'C:\Users\data science\Desktop\OTT_project\trained_models\genre_encoder.joblib')
label_encoder_age_rating = load(r'C:\Users\data science\Desktop\OTT_project\trained_models\age_rating_encoder.joblib')
label_encoder_platform = load(r'C:\Users\data science\Desktop\OTT_project\trained_models\platform_encoder.joblib')

def predict_platform():
    genre_input = genre_entry.get()
    duration_input = duration_entry.get()
    age_rating_input = age_rating_entry.get()

    try:
        # Validate and convert duration to integer
        duration_input = int(duration_input)
        # Encode the inputs
        genre_encoded = label_encoder_genre.transform([genre_input])[0]
        age_rating_encoded = label_encoder_age_rating.transform([age_rating_input])[0]
        
        # Prepare the input for prediction
        input_features = pd.DataFrame({
            'genre_encoded': [genre_encoded],
            'duration_min': [duration_input],
            'age_rating_encoded': [age_rating_encoded]
        })
        
        # Predict the platform using the classifier
        predicted_platform_encoded = clf.predict(input_features)
        predicted_platform = label_encoder_platform.inverse_transform(predicted_platform_encoded)
        
        # Display the predicted platform in the GUI label
        result_label.config(text=f"Best Platform for the new movie: {predicted_platform[0]}")
    except ValueError as e:
        result_label.config(text=f"Error: {e}. Please ensure your inputs match the dataset's categories.")
    except Exception as e:
        result_label.config(text=f"An error occurred: {e}")

# Setup the GUI
root = tk.Tk()
root.title("Movie Platform Predictor")

# Genre input
tk.Label(root, text="Enter Genre:").grid(row=0, column=0, padx=10, pady=10)
genre_entry = tk.Entry(root)
genre_entry.grid(row=0, column=1, padx=10, pady=10)

# Duration input
tk.Label(root, text="Enter Duration (min):").grid(row=1, column=0, padx=10, pady=10)
duration_entry = tk.Entry(root)
duration_entry.grid(row=1, column=1, padx=10, pady=10)

# Age rating input
tk.Label(root, text="Enter Age Rating:").grid(row=2, column=0, padx=10, pady=10)
age_rating_entry = tk.Entry(root)
age_rating_entry.grid(row=2, column=1, padx=10, pady=10)

# Prediction button
predict_button = tk.Button(root, text="Predict Platform", command=predict_platform)
predict_button.grid(row=3, column=0, columnspan=2, pady=10)

# Result display label
result_label = tk.Label(root, text="Prediction will appear here")
result_label.grid(row=4, column=0, columnspan=2, padx=10, pady=10)

root.mainloop()
